# 查询耗时测试

为什么百度百科的公园数据和动物数据，summaryIndex 生成回答耗时不同：

- 公园数据似乎和文档数无关
- 动物数据和文档数线性相关


## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    # model="qwen2", 
    model="glm4", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 3.23 s, sys: 416 ms, total: 3.65 s
Wall time: 3.26 s


## 公园数据测试

In [2]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

items=[
    "颐和园",
    "北海公园",
    "故宫",
    "八达岭长城",
    "天坛公园",
    "恭王府",
    "圆明园",
    "景山公园",
    "中山公园",
    "陶然亭公园",
]

documents_attractions = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents_attractions)

CPU times: user 1.74 s, sys: 25.9 ms, total: 1.76 s
Wall time: 2.64 s


10

In [3]:
%%time

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SummaryIndex

parser = SentenceSplitter(chunk_size=1024,chunk_overlap=20)
nodes = parser.get_nodes_from_documents(documents_attractions)
summary_index_attractions = SummaryIndex(nodes)

len(nodes)

CPU times: user 370 ms, sys: 90 µs, total: 370 ms
Wall time: 370 ms


171

In [4]:
%%time

# 创建索引器

summary_retriever_attractions = summary_index_attractions.as_retriever()

CPU times: user 26 µs, sys: 2 µs, total: 28 µs
Wall time: 30.3 µs


In [5]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True,
)

query_engine = RetrieverQueryEngine(
    retriever=summary_retriever_attractions,
    response_synthesizer=response_synthesizer,
)

CPU times: user 133 µs, sys: 12 µs, total: 145 µs
Wall time: 147 µs


In [6]:
%%time

response = query_engine.query("介绍下颐和园的历史")
response.print_response_stream()

颐和园位于北京市海淀区，是中国著名的皇家园林。其历史始于金代，但现在的规模和格局主要是在清朝乾隆年间形成的。1750年，为庆祝皇太后六十寿辰，清乾隆皇帝下令在昆明湖畔建造“清漪园”。然而，1860年英法联军入侵北京时，清漪园遭到严重破坏。1886年，慈禧太后重建园林，并更名为“颐和园”，成为清朝皇室的夏宫。1900年八国联军侵华战争期间，颐和园再次遭受破坏。1912年中华民国成立后，颐和园被辟为公园，对外开放。新中国成立后，政府对颐和园进行了大规模的修复和保护工作，使其成为中国现存规模最大、保存最完整的皇家园林之一。1998年，颐和园被列入世界文化遗产名录。CPU times: user 898 ms, sys: 35.1 ms, total: 933 ms
Wall time: 9min 34s
